# rian-kh, 06/09/23

# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [262]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    
    
    population_by_region = [] 
    win_loss_by_region = []

    citiesDF = cities[["Metropolitan area", 'Population (2016 est.)[8]', 'NHL']].set_index("Metropolitan area")
    nhlDF = nhl_df[['team', 'W', 'L', 'year']].set_index('team')
   
    
    # NHL dataframe: Drop 'division' rows 
    for index in nhlDF.index.unique():
        if index.split(' ')[-1] == 'Division':
            nhlDF = nhlDF.drop(index)
        
    # NHL dataframe: Get rid of asterisks
    for index in nhlDF.index:
        if index[-1] == '*':
            nhlDF = nhlDF.rename({index:index[:index.find('*')]})
            
    # Set values to floats in both dataframes
    nhlDF['W'] = nhlDF['W'].astype('float')
    nhlDF['L'] = nhlDF['L'].astype('float')
    nhlDF['year'] = nhlDF['year'].astype('float')
    citiesDF['Population (2016 est.)[8]'] = citiesDF['Population (2016 est.)[8]'].astype('float')
    
    # Only get data from 2018
    nhlDF = nhlDF[nhlDF['year'] == 2018]
    
    # Loop through each city
    for city in citiesDF.index:
        
        # Win/Loss sums for all teams in current city
        sumWins = 0;
        sumLosses = 0;
        
        
        # Loop through each team
        for team in nhlDF.index.unique():
             
            splitName = team.split(' ')
            
            # Check if the last word of each team matches
            # one of the NHL teams for each city, and add
            # to win/loss sums per team
            if splitName[-1] in citiesDF.loc[city]['NHL']:
                sumWins += nhlDF.loc[team]['W'].sum()
                sumLosses += nhlDF.loc[team]['L'].sum()
            
        
        # If the current team was a part of a city (losses > 0),
        # add to the population the current city's pop. and to the
        # win/loss list the current city's ratio
        if sumLosses != 0:
            population_by_region.append(citiesDF.loc[city]['Population (2016 est.)[8]'])
            win_loss_by_region.append(sumWins / (sumWins+sumLosses))
            
  
    
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012308996455744285

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [283]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    population_by_region = [] 
    win_loss_by_region = []

    citiesDF = cities[["Metropolitan area", 'Population (2016 est.)[8]', 'NBA']].set_index("Metropolitan area")
    nbaDF = nba_df[['team', 'W', 'L', 'year']].set_index('team')
    
    # NBA dataframe: Drop 'division' rows 
    for index in nbaDF.index.unique():
        if index.split(' ')[-1] == 'Division':
            nbaDF = nbaDF.drop(index)
        

    
    # NBA dataframe: Get rid of parentheses and asterisks
    for index in nbaDF.index:
        newIndex = re.sub('(\*\s\(\d*\)$)|(\*$)|(\s\(\d*\)$)', '', index)
        nbaDF = nbaDF.rename({index:newIndex})
        
    
            
    # Set values to floats in both dataframes
    nbaDF['W'] = nbaDF['W'].astype('float')
    nbaDF['L'] = nbaDF['L'].astype('float')
    nbaDF['year'] = nbaDF['year'].astype('float')
    citiesDF['Population (2016 est.)[8]'] = citiesDF['Population (2016 est.)[8]'].astype('float')
    
    # Only get data from 2018
    nbaDF = nbaDF[nbaDF['year'] == 2018]
    
    
    # Loop through each city
    for city in citiesDF.index:
        
        # Win/Loss sums for all teams in current city
        sumWins = 0;
        sumLosses = 0;
        
        
        # Loop through each team
        for team in nbaDF.index.unique():
    
            splitName = team.split(' ')
            
            # Check if the last word of each team matches
            # one of the NHL teams for each city, and add
            # to win/loss sums per team
            if splitName[-1] in citiesDF.loc[city]['NBA']:
                sumWins += nbaDF.loc[team]['W'].sum()
                sumLosses += nbaDF.loc[team]['L'].sum()
        
        
        # If the current team was a part of a city (losses > 0),
        # add to the population the current city's pop. and to the
        # win/loss list the current city's ratio
        if sumLosses != 0:
            population_by_region.append(citiesDF.loc[city]['Population (2016 est.)[8]'])
            win_loss_by_region.append(sumWins / (sumWins+sumLosses))

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.1765716025284462

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [313]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    population_by_region = [] 
    win_loss_by_region = []

    citiesDF = cities[["Metropolitan area", 'Population (2016 est.)[8]', 'MLB']].set_index("Metropolitan area")
    mlbDF = mlb_df[['team', 'W', 'L', 'year']].set_index('team')
       
    # Set values to floats in both dataframes
    mlbDF['W'] = mlbDF['W'].astype('float')
    mlbDF['L'] = mlbDF['L'].astype('float')
    mlbDF['year'] = mlbDF['year'].astype('float')
    citiesDF['Population (2016 est.)[8]'] = citiesDF['Population (2016 est.)[8]'].astype('float')
    
    # Only get data from 2018
    mlbDF = mlbDF[mlbDF['year'] == 2018]
    
    # Loop through each city
    for city in citiesDF.index:
        
        # Win/Loss sums for all teams in current city
        sumWins = 0;
        sumLosses = 0;
        
        
        # Loop through each team
        for team in mlbDF.index.unique():
    
            splitName = team.split(' ')
            
            
            # HARD CODE: This code groups the city-team by 
            # last word, BUT "Red Sox" is Boston while
            # "White Sox" is Chicago.
            
            if team == "Boston Red Sox" and city == "Chicago" or team == "Chicago White Sox" and city == "Boston":
                continue
            
            
            # Check if the last word of each team matches
            # one of the NHL teams for each city, and add
            # to win/loss sums per team
            
            if splitName[-1] in citiesDF.loc[city]['MLB']:
                sumWins += mlbDF.loc[team]['W'].sum()
                sumLosses += mlbDF.loc[team]['L'].sum()
        
        
        # If the current team was a part of a city (losses > 0),
        # add to the population the current city's pop. and to the
        # win/loss list the current city's ratio
        if sumLosses != 0:
            population_by_region.append(citiesDF.loc[city]['Population (2016 est.)[8]'])
            win_loss_by_region.append(sumWins / (sumWins+sumLosses))            
    
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [265]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    population_by_region = [] 
    win_loss_by_region = []

    citiesDF = cities[["Metropolitan area", 'Population (2016 est.)[8]', 'NFL']].set_index("Metropolitan area")
    nflDF = nfl_df[['team', 'W', 'L', 'year']].set_index('team')
      
    # Get rid of "___ East/North/etc" entries (?)
    nflDF = nflDF[nflDF['W'].str.isdigit()]
    

    # NFL dataframe: Get rid of +es and *s
    for index in nflDF.index:
        newIndex = re.sub('\W$', '', index)
        nflDF = nflDF.rename({index:newIndex})
        
    # Set values to floats in both dataframes
    nflDF['W'] = nflDF['W'].astype('float')
    nflDF['L'] = nflDF['L'].astype('float')
    nflDF['year'] = nflDF['year'].astype('float')
    citiesDF['Population (2016 est.)[8]'] = citiesDF['Population (2016 est.)[8]'].astype('float')
    
    # Only get data from 2018
    nflDF = nflDF[nflDF['year'] == 2018]
    
    # Loop through each city
    for city in citiesDF.index:
        
        # Win/Loss sums for all teams in current city
        sumWins = 0;
        sumLosses = 0;
        
        
        # Loop through each team
        for team in nflDF.index.unique():
    
            splitName = team.split(' ')
            
            # Check if the last word of each team matches
            # one of the NHL teams for each city, and add
            # to win/loss sums per team
            if splitName[-1] in citiesDF.loc[city]['NFL']:
                sumWins += nflDF.loc[team]['W'].sum()
                sumLosses += nflDF.loc[team]['L'].sum()
        
        
        # If the current team was a part of a city (losses > 0),
        # add to the population the current city's pop. and to the
        # win/loss list the current city's ratio
        if sumLosses != 0:
            population_by_region.append(citiesDF.loc[city]['Population (2016 est.)[8]'])
            win_loss_by_region.append(sumWins / (sumWins+sumLosses))

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004922112149349414

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [229]:
# INCOMPLETE


import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    
    # Get win/loss of all cities in a sport as a decimal
    # Use cols as the p_value thing
    
    
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    
    print(p_values)
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

     NFL  NBA  NHL  MLB
NFL  NaN  NaN  NaN  NaN
NBA  NaN  NaN  NaN  NaN
NHL  NaN  NaN  NaN  NaN
MLB  NaN  NaN  NaN  NaN


AssertionError: The NBA-NHL p-value should be around 0.02